In [4]:
import pandas as pd

In [5]:
# read the scraped recommendations
df_recs_bp = pd.read_csv('data/bestprice_recs/bestprice_recs.csv')
# read the products with their details
df_products = pd.read_pickle('data/product_details/df_preproc.pkl')
# read our recommendations
df_recs = pd.read_pickle('data/recommendations/df_recos_brand_1-5_category_1-5.pkl')

In [6]:
df_recs_bp.head()

,product_id,rec_id,rec_url
0,2154993913,2154526132,https://www.bestprice.gr/item/2154526132/fineb...
1,2154993913,2152156746,https://www.bestprice.gr/item/2152156746/nokia...
2,2154993913,2154628447,https://www.bestprice.gr/item/2154628447/samsu...
3,2154993913,2154745683,https://www.bestprice.gr/item/2154745683/samsu...
4,2154993913,2154273618,https://www.bestprice.gr/item/2154273618/plant...


In [7]:
df_products.head(2)

,product_id,url,Title,Category2,SubCategory2,brand_name2,Title_lc2
0,2155531584,https://www.bestprice.gr/item/2155531584/samsu...,Samsung Galaxy S10+ 128GB Dual,mobiles_cat,mobile_phone_subcat,samsung_brand,samsung_brand galaxy s10+ 128gb dual mobiles_c...
1,2155621215,https://www.bestprice.gr/item/2155621215/samsu...,Samsung Galaxy A70 6GB 128GB Dual,mobiles_cat,mobile_phone_subcat,samsung_brand,samsung_brand galaxy a70 6gb 128gb dual mobile...


In [8]:
df_recs.head(2)

,doc_id,term,term_id,tfidf,product_id,brand_name2,Category2,SubCategory2,Minhash_id,Minhash,recs,Brands,Categories,Subcategories,N_Brands,N_Categories,N_Subcategories
0,0,"[samsung_brand, galaxy, s10, 128gb, dual, mobi...","[28661, 19239, 27889, 1587, 14268, 22300, 22299]","[0.5190237210992299, 0.3732597244316061, 0.557...",2155531584,samsung_brand,mobiles_cat,mobile_phone_subcat,m0,<datasketch.weighted_minhash.WeightedMinHash o...,"[m333, m0, m12, m1, m3, m23, m17, m22, m21, m732]","[ulefone_brand, samsung_brand, samsung_brand, ...","[mobiles_cat, mobiles_cat, mobiles_cat, mobile...","[mobile_phone_subcat, mobile_phone_subcat, mob...",3,1,1
1,1,"[samsung_brand, galaxy, 128gb, dual, mobiles_c...","[28661, 19239, 1587, 14268, 22300, 22299, 8666...","[0.43980326188651964, 0.3162877488300745, 0.30...",2155621215,samsung_brand,mobiles_cat,mobile_phone_subcat,m1,<datasketch.weighted_minhash.WeightedMinHash o...,"[m0, m12, m1, m3, m125, m23, m22, m50, m210, m...","[samsung_brand, samsung_brand, samsung_brand, ...","[mobiles_cat, mobiles_cat, mobiles_cat, mobile...","[mobile_phone_subcat, mobile_phone_subcat, mob...",5,1,1


In [9]:
df_products['product_id'] = df_products['product_id'].astype('str')
df_recs_bp['product_id'] = df_recs_bp['product_id'].astype('str')
df_recs_bp['rec_id'] = df_recs_bp['rec_id'].astype('str')

In [10]:
# In df_recs_bp keep only recommended products for which we have the details
# Note: we have details for 3854/4582 recommended products.
df_recs_bp = df_recs_bp[df_recs_bp.rec_id.isin(df_products.product_id.unique())]

In [11]:
# keep a matching between 'product_id' & 'Minhash_id'
df_m = df_recs[['product_id', 'Minhash_id']]
df_m['product_id'] = df_m['product_id'].astype('str')

C:\Users\Dimos\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
df_m.head(1)

,product_id,Minhash_id
0,2155531584,m0


In [13]:
# add the 'Minhash_id' to each scraped recommendation
df_recs_bp = df_recs_bp.merge(df_recs[['product_id', 'Minhash_id']], left_on='rec_id', right_on='product_id', how='left')

In [14]:
# add the category, subcategory and brand of each recommended product
df_recs_bp = df_recs_bp.merge(df_products[['product_id', 'Category2', 'SubCategory2', 'brand_name2']], left_on='rec_id', right_on='product_id', how='left')

In [15]:
df_recs_bp.head()

,product_id_x,rec_id,rec_url,product_id_y,Minhash_id,product_id,Category2,SubCategory2,brand_name2
0,2154993913,2154526132,https://www.bestprice.gr/item/2154526132/fineb...,2154526132,m893,2154526132,mobiles_cat,bluetooth_subcat,fineblue_brand
1,2154993913,2152156746,https://www.bestprice.gr/item/2152156746/nokia...,2152156746,m1316,2152156746,mobiles_cat,bluetooth_subcat,nokia_brand
2,2154993913,2154628447,https://www.bestprice.gr/item/2154628447/samsu...,2154628447,m947,2154628447,mobiles_cat,bluetooth_subcat,samsung_brand
3,2154993913,2154745683,https://www.bestprice.gr/item/2154745683/samsu...,2154745683,m941,2154745683,mobiles_cat,bluetooth_subcat,samsung_brand
4,2154993913,2154273618,https://www.bestprice.gr/item/2154273618/plant...,2154273618,m963,2154273618,mobiles_cat,bluetooth_subcat,plantronics_brand


In [16]:
# group by product and convert recommendations to lists
#df_recs_bp2 = df_recs_bp.groupby(['product_id_x'])['Minhash_id'].apply(list).reset_index(name='bp_recs')
df_recs_bp2 = df_recs_bp.groupby(['product_id_x'], as_index=False)['Minhash_id', 'Category2', 'SubCategory2', 'brand_name2'].agg(lambda x: list(x))
df_recs_bp2.columns = ['product_id', 'bp_recs', 'bp_categories', 'bp_subcategories', 'bp_brands']
df_recs_bp2['bp_recs'] = df_recs_bp2['bp_recs'].apply(lambda x: x[0:10])
df_recs_bp2.head(2)

,product_id,bp_recs,bp_categories,bp_subcategories,bp_brands
0,2150590477,"[m25887, m25867, m25874, m25882, m25891, m2587...","[photograph_cat, photograph_cat, photograph_ca...","[dslr_subcat, dslr_subcat, dslr_subcat, dslr_s...","[nikon_brand, nikon_brand, nikon_brand, nikon_..."
1,2150590749,"[m25749, m25761, m25740, m25768, m25752, m2573...","[photograph_cat, photograph_cat, photograph_ca...","[compact_subcat, compact_subcat, compact_subca...","[sony_brand, sony_brand, sony_brand, easypix_b..."


In [17]:
df_recs_bp2.shape

(1555, 5)

In [18]:
df_recs.head(1)

,doc_id,term,term_id,tfidf,product_id,brand_name2,Category2,SubCategory2,Minhash_id,Minhash,recs,Brands,Categories,Subcategories,N_Brands,N_Categories,N_Subcategories
0,0,"[samsung_brand, galaxy, s10, 128gb, dual, mobi...","[28661, 19239, 27889, 1587, 14268, 22300, 22299]","[0.5190237210992299, 0.3732597244316061, 0.557...",2155531584,samsung_brand,mobiles_cat,mobile_phone_subcat,m0,<datasketch.weighted_minhash.WeightedMinHash o...,"[m333, m0, m12, m1, m3, m23, m17, m22, m21, m732]","[ulefone_brand, samsung_brand, samsung_brand, ...","[mobiles_cat, mobiles_cat, mobiles_cat, mobile...","[mobile_phone_subcat, mobile_phone_subcat, mob...",3,1,1


In [19]:
# keep only the same information of our recommendations
df_recs = df_recs[['product_id', 'recs', 'Brands', 'Categories', 'Subcategories']]
# convert 'product_id' to int
#df_recs['product_id'] = df_recs['product_id'].astype('int')

df_recs.head(2)

,product_id,recs,Brands,Categories,Subcategories
0,2155531584,"[m333, m0, m12, m1, m3, m23, m17, m22, m21, m732]","[ulefone_brand, samsung_brand, samsung_brand, ...","[mobiles_cat, mobiles_cat, mobiles_cat, mobile...","[mobile_phone_subcat, mobile_phone_subcat, mob..."
1,2155621215,"[m0, m12, m1, m3, m125, m23, m22, m50, m210, m...","[samsung_brand, samsung_brand, samsung_brand, ...","[mobiles_cat, mobiles_cat, mobiles_cat, mobile...","[mobile_phone_subcat, mobile_phone_subcat, mob..."


In [20]:
# create a df with all the recs(scraped & our) 
df_recs_all = df_recs_bp2.merge(df_recs, left_on='product_id', right_on='product_id', how='left')

In [21]:
df_recs_all.head(2)

,product_id,bp_recs,bp_categories,bp_subcategories,bp_brands,recs,Brands,Categories,Subcategories
0,2150590477,"[m25887, m25867, m25874, m25882, m25891, m2587...","[photograph_cat, photograph_cat, photograph_ca...","[dslr_subcat, dslr_subcat, dslr_subcat, dslr_s...","[nikon_brand, nikon_brand, nikon_brand, nikon_...","[m25887, m25881, m25868, m25866, m25925, m2587...","[nikon_brand, nikon_brand, nikon_brand, nikon_...","[photograph_cat, photograph_cat, photograph_ca...","[dslr_subcat, dslr_subcat, dslr_subcat, dslr_s..."
1,2150590749,"[m25749, m25761, m25740, m25768, m25752, m2573...","[photograph_cat, photograph_cat, photograph_ca...","[compact_subcat, compact_subcat, compact_subca...","[sony_brand, sony_brand, sony_brand, easypix_b...","[m25833, m25815, m25818, m25835, m25820, m2581...","[panasonic_brand, panasonic_brand, panasonic_b...","[photograph_cat, photograph_cat, photograph_ca...","[compact_subcat, compact_subcat, compact_subca..."


In [22]:
# count how many recs are the same
df_recs_all['N_common_recs'] = df_recs_all.apply(lambda x: len(set(x['bp_recs']).intersection(set(x['recs']))), axis=1)

In [23]:
# count number of categories in bp recs
df_recs_all['N_Categories_bp'] = df_recs_all.apply(lambda x: len(set(x['bp_categories'])), axis=1)
# count number of subcategories in bp recs
df_recs_all['N_Subcategories_bp'] = df_recs_all.apply(lambda x: len(set(x['bp_subcategories'])), axis=1)
# count number of brands in bp recs
df_recs_all['N_brands_bp'] = df_recs_all.apply(lambda x: len(set(x['bp_brands'])), axis=1)

In [24]:
# count number of categories in our recs
df_recs_all['N_Categories'] = df_recs_all.apply(lambda x: len(set(x['Categories'])), axis=1)
# count number of subcategories in our recs
df_recs_all['N_Subcategories'] = df_recs_all.apply(lambda x: len(set(x['Subcategories'])), axis=1)
# count number of brands in our recs
df_recs_all['N_brands'] = df_recs_all.apply(lambda x: len(set(x['Brands'])), axis=1)

In [25]:
# count the number of bestprice recs 
df_recs_all['N_recs_bp'] = df_recs_all.apply(lambda x: len(x['bp_recs']), axis=1)
# count the number of our recs 
df_recs_all['N_recs'] = df_recs_all.apply(lambda x: len(x['recs']), axis=1)

In [26]:
# remove some products that have 10<recommendations<13 in bestprice
df_recs_all = df_recs_all[df_recs_all.N_recs_bp>=10]
df_recs_all = df_recs_all[df_recs_all.N_recs_bp<13]
# remove some products that have less than 10 recommendations
df_recs_all = df_recs_all[df_recs_all.N_recs>=10]

!!!!!! Maybe remove cases where N_Categories>1

In [27]:
# remove cases where N_Categories>1
#df_recs_all = df_recs_all[df_recs_all.N_Categories<=1]

In [28]:
df_recs_all.head()

,product_id,bp_recs,bp_categories,bp_subcategories,bp_brands,recs,Brands,Categories,Subcategories,N_common_recs,N_Categories_bp,N_Subcategories_bp,N_brands_bp,N_Categories,N_Subcategories,N_brands,N_recs_bp,N_recs
0,2150590477,"[m25887, m25867, m25874, m25882, m25891, m2587...","[photograph_cat, photograph_cat, photograph_ca...","[dslr_subcat, dslr_subcat, dslr_subcat, dslr_s...","[nikon_brand, nikon_brand, nikon_brand, nikon_...","[m25887, m25881, m25868, m25866, m25925, m2587...","[nikon_brand, nikon_brand, nikon_brand, nikon_...","[photograph_cat, photograph_cat, photograph_ca...","[dslr_subcat, dslr_subcat, dslr_subcat, dslr_s...",5,1,1,2,1,1,1,10,10
2,2151123845,"[m18357, m17890, m17907, m18180, m17613, m1812...","[desktop_pc_cat, desktop_pc_cat, desktop_pc_ca...","[desktop_monitors_subcat, desktop_monitors_sub...","[philips_brand, aoc_brand, aoc_brand, dell_bra...","[m18781, m18877, m18796, m18785, m18852, m1880...","[hp_brand, hp_brand, hp_brand, hp_brand, hp_br...","[desktop_pc_cat, desktop_pc_cat, desktop_pc_ca...","[desktop_monitors_subcat, desktop_monitors_sub...",0,1,1,8,1,1,1,10,10
3,2151124109,"[m17601, m17782, m17897, m18011, m17948, m1835...","[desktop_pc_cat, desktop_pc_cat, desktop_pc_ca...","[desktop_monitors_subcat, desktop_monitors_sub...","[lg_brand, samsung_brand, aoc_brand, asus_bran...","[m17720, m17648, m17564, m17548, m17568, m1752...","[lg_brand, lg_brand, lg_brand, lg_brand, lg_br...","[desktop_pc_cat, desktop_pc_cat, desktop_pc_ca...","[desktop_monitors_subcat, desktop_monitors_sub...",1,1,1,6,1,1,1,10,10
4,2151181409,"[m20160, m20146, m20024, m20145, m20162, m2020...","[desktop_pc_cat, desktop_pc_cat, desktop_pc_ca...","[desktop_rams_subcat, desktop_rams_subcat, des...","[corsair_brand, corsair_brand, corsair_brand, ...","[m20406, m20204, m21013, m21009, m20145, m2097...","[corsair_brand, corsair_brand, crucial_brand, ...","[desktop_pc_cat, desktop_pc_cat, desktop_pc_ca...","[desktop_rams_subcat, desktop_rams_subcat, des...",2,1,1,1,1,1,3,10,10
5,2151181415,"[m20146, m20399, m20218, m20400, m20160, m2040...","[desktop_pc_cat, desktop_pc_cat, desktop_pc_ca...","[desktop_rams_subcat, desktop_rams_subcat, des...","[corsair_brand, corsair_brand, corsair_brand, ...","[m20880, m20400, m20064, m21351, m20398, m2007...","[mushkin_brand, corsair_brand, corsair_brand, ...","[desktop_pc_cat, desktop_pc_cat, desktop_pc_ca...","[desktop_rams_subcat, desktop_rams_subcat, des...",3,1,1,1,1,1,3,10,10


In [29]:
df_recs_all[df_recs_all.N_common_recs==0].shape

(914, 18)

In [30]:
# df_recos_category_2-0.pkl

In [31]:
df_recs_descr = df_recs_all.describe()
df_recs_descr = df_recs_descr.round(2)
df_recs_descr

,N_common_recs,N_Categories_bp,N_Subcategories_bp,N_brands_bp,N_Categories,N_Subcategories,N_brands,N_recs_bp,N_recs
count,1195.00,1195.0,1195.0,1195.00,1195.00,1195.00,1195.00,1195.0,1195.0
mean,0.38,1.0,1.0,5.16,1.10,1.24,2.18,10.0,10.0
std,0.83,0.0,0.0,1.98,0.34,0.59,1.69,0.0,0.0
min,0.00,1.0,1.0,1.00,1.00,1.00,1.00,10.0,10.0
25%,0.00,1.0,1.0,4.00,1.00,1.00,1.00,10.0,10.0
50%,0.00,1.0,1.0,5.00,1.00,1.00,1.00,10.0,10.0
75%,0.00,1.0,1.0,7.00,1.00,1.00,3.00,10.0,10.0
max,6.00,1.0,1.0,11.00,4.00,6.00,9.00,10.0,10.0


In [282]:
df_recs_descr = df_recs_all.describe()
df_recs_descr = df_recs_descr.round(2)
df_recs_descr

,N_common_recs,N_Categories_bp,N_Subcategories_bp,N_brands_bp,N_Categories,N_Subcategories,N_brands,N_recs_bp,N_recs
count,1182.00,1182.0,1182.0,1182.00,1182.00,1182.00,1182.00,1182.00,1182.0
mean,0.40,1.0,1.0,5.16,1.10,1.24,2.18,11.35,10.0
std,0.85,0.0,0.0,1.97,0.34,0.59,1.69,0.73,0.0
min,0.00,1.0,1.0,1.00,1.00,1.00,1.00,10.00,10.0
25%,0.00,1.0,1.0,4.00,1.00,1.00,1.00,11.00,10.0
50%,0.00,1.0,1.0,5.00,1.00,1.00,1.00,12.00,10.0
75%,0.00,1.0,1.0,7.00,1.00,1.00,3.00,12.00,10.0
max,6.00,1.0,1.0,11.00,4.00,6.00,9.00,12.00,10.0
